In [17]:
import urllib.parse
import geopy
from geopy.geocoders import Nominatim
from searchengine.trip_finder import SearchEngine

def get_coordinates(address):
    geolocator = Nominatim(user_agent="streamlit_app", timeout=10)
    location = geolocator.geocode(f"{address}, Rhône, France")
    if location:
        return location.latitude, location.longitude

departure_address = "323 Chemin des Ferratières Charly"
arrival_address = "5 Av. Pierre Mendès France, 69500 Bron"
    
dep_lat, dep_lon = get_coordinates(departure_address)
arr_lat, arr_lon = get_coordinates(arrival_address)
            
if dep_lat and dep_lon and arr_lat and arr_lon:
    from_coord = f"{dep_lon};{dep_lat}"
    to_coord = f"{arr_lon};{arr_lat}"
    encoded_from = urllib.parse.quote(from_coord)
    encoded_to = urllib.parse.quote(to_coord)
    
    tcl_url = f"https://www.tcl.fr/itineraires?date=now&pmr=0&from={encoded_from}&to={encoded_to}"
    print(tcl_url)

https://www.tcl.fr/itineraires?date=now&pmr=0&from=4.8019631%3B45.6428848&to=4.9151265%3B45.7225931
